In [1]:
# Leitura do banco de dados SQLite
import pandas as pd
from sqlalchemy import create_engine
DATABASE_FILE = "dados_sensores.db"
engine = create_engine(f'sqlite:///{DATABASE_FILE}')

nome_tabela = "leituras_sensores"
query = f"SELECT * FROM {nome_tabela}"
try:
    df = pd.read_sql_query(query, engine)
    print("Dados lidos com sucesso:")
    print(df.head())
except Exception as e:
    print(f"\nOcorreu um erro ao ler o banco de dados: {e}")

Dados lidos com sucesso:
   id           tag description                   timestamp      value
0   1  LIC_22311020   NÍVEL T01  2015-10-07 19:53:06.426635  60.242100
1   2  LIC_22311020   NÍVEL T01  2015-10-07 21:55:39.881011  60.715893
2   3  LIC_22311020   NÍVEL T01  2015-10-07 22:21:52.142013  59.573433
3   4  LIC_22311020   NÍVEL T01  2015-10-07 22:36:09.136001  59.805626
4   5  LIC_22311020   NÍVEL T01  2015-10-07 23:11:18.985000  60.222218


In [ ]:
# Análise exploratória dos dados quantidade de leituras por tag e descrição
#print(df['description'].value_counts())
#print(df['tag'].value_counts())
df[df['tag'] == 'PX_22311020.RB']
df['timestamp'] = pd.to_datetime(df['timestamp'])
diferencas = df['timestamp'].diff()
diferencas.mean()

Timedelta('0 days 00:12:51.375899811')

In [17]:
# Obtendo o máximo e mínimo de data por tag
""" Essa etapa é importante para padronizar o timestamp de cada tag.
"""
list_distinct_tags = df['tag'].unique()

df_dates = pd.DataFrame(columns=['tag', 'max_timestamp', 'min_timestamp'])
for tag in list_distinct_tags:
    df_tag = df[df['tag'] == tag]
    max_timestamp = df_tag['timestamp'].max()
    min_timestamp = df_tag['timestamp'].min()
    df_dates = pd.concat(
        [df_dates, pd.DataFrame({'tag': [tag], 'max_timestamp': [max_timestamp], 'min_timestamp': [min_timestamp]})],
        ignore_index=True
    )
df_dates['max_timestamp'] = pd.to_datetime(df_dates['max_timestamp'])
df_dates['min_timestamp'] = pd.to_datetime(df_dates['min_timestamp'])
df_dates




,tag,max_timestamp,min_timestamp
0,LIC_22311020,2025-10-07 19:46:32.369003,2015-10-07 19:53:06.426635
1,LIC_22311020.ES,2025-10-07 19:47:04.314010,2015-10-07 19:53:10.512130
2,LIC_22311020.MV,2025-10-07 19:53:14.910888,2015-10-07 19:53:14.910888
3,LIC_22311020.SV,2025-10-07 19:47:04.314010,2015-10-07 19:53:23.740142
4,PX_22311020.C1,2025-10-07 19:52:54.256011,2025-07-02 12:07:58.167007
5,PX_22311020.C2,2025-10-07 19:50:52.572006,2025-07-02 12:04:23.199005
6,PX_22311020.DEVPOS,2025-10-07 19:52:54.261001,2023-03-09 19:09:05.682006
7,PX_22311020.DIFF,2025-10-07 19:53:54.298263,2025-08-26 19:22:48.026000
8,PX_22311020.OUT,2025-10-07 19:53:55.097000,2025-07-02 12:11:24.026000
9,PX_22311020.PRESS,2025-10-07 19:52:54.256011,2023-03-09 19:13:08.042999


In [28]:
# tempo médio entre leituras por tag
df['timestamp'] = pd.to_datetime(df['timestamp'])
df_diff = pd.DataFrame(columns=['tag', 'mean_diff'])
for tag in list_distinct_tags:
    df_tag = df[df['tag'] == tag]
    diferencas = df_tag['timestamp'].diff()
    #print(f"Tag: {tag}, Tempo médio entre leituras: {diferencas.mean()}")
    df_diff = pd.concat([df_diff, pd.DataFrame({'tag': [tag], 'mean_diff': [diferencas.mean()]})], ignore_index=True)

df_diff

C:\Users\wateu\AppData\Local\Temp\ipykernel_5020\2160075855.py:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_diff = pd.concat([df_diff, pd.DataFrame({'tag': [tag], 'mean_diff': [diferencas.mean()]})], ignore_index=True)


,tag,mean_diff
0,LIC_22311020,0 days 00:59:56.303707098
1,LIC_22311020.ES,0 days 01:27:49.887525703
2,LIC_22311020.MV,0 days 00:45:18.980013783
3,LIC_22311020.SV,0 days 01:27:23.708599
4,PX_22311020.C1,0 days 00:44:06.740978597
5,PX_22311020.C2,0 days 00:44:59.450842054
6,PX_22311020.DEVPOS,0 days 01:03:04.913298601
7,PX_22311020.DIFF,0 days 00:44:00.484561645
8,PX_22311020.OUT,0 days 00:44:28.534138686
9,PX_22311020.PRESS,0 days 00:57:37.421124204
